#### Training the source network on the poems (frozen weights from songs)

In [1]:
import text_processing
import torch
import torch.nn.functional as F
from imp import reload
import models
reload(models)
from text_processing import read_split_file_lyrics, BatchWrapper, generate_iterators_lyrics, generate_iterators
from functions import *
import numpy as np
import pandas as pd
from models import *

In [2]:
print("Creating iterators.")
#Create iterators
emotions= ['anger', 'anticipation', 'fear', 'joy', 'love', 'optimism', 'pess', 'sad']
torch.cuda.empty_cache() 
batch_size = 10
train_iter_p, val_iter_p, test_iter_p, TEXT = generate_iterators("split_used_for_baseline/poems_train.csv", "split_used_for_baseline/poems_val.csv",
                                                                  "split_used_for_baseline/poems_test.csv", batch_size = batch_size, device="cuda")

Creating iterators.


In [3]:
dropout_src = 0.350748
learning_rate_src = 0.000146
hidden_dim_src = 183
v_dim_src = 21
label_size_src = 2
label_size = 8
embedding_dim_src = 300


train_batch = BatchWrapper(train_iter_p, "text", emotions)
valid_batch = BatchWrapper(val_iter_p, "text", emotions)
test_batch = BatchWrapper(test_iter_p, "text", emotions)

m_source_p = BiLSTMSourceNet(vocab_size = len(TEXT.vocab), embedding_dim = embedding_dim_src, 
                               hidden_dim = hidden_dim_src, label_size=label_size_src, v_dim = v_dim_src, 
                               pretrained_vec=TEXT.vocab.vectors, use_gpu = True, dropout = dropout_src)

m_source_p.to("cuda")
src_model_dict = torch.load("SourceNetModelDict_Final.pth")
src_model_dict['embeddings.weight'] = TEXT.vocab.vectors
# src_model_dict['decoder.bias'] = torch.ones(8)
# src_model_dict['decoder.weight'] = torch.ones(8, hidden_dim*2)
m_source_p.load_state_dict(src_model_dict)

for param in m_source_p.parameters():
        param.requires_grad = False

In [4]:
dropout_min = 0.2
dropout_max = 0.8

learning_rate_min = 0.0001
learning_rate_max = 0.01

hidden_dim_min = 50
hidden_dim_max = 300

v_dim_min = 5
v_dim_max = 30
embedding_dim = 300

dropouts = []
learning_rates = []
hidden_dims = []
v_dims = []
dev_accuracies = []
test_accuracies = []

best_dev_acc = 0
best_test_acc = 0
best_hidden_dim = 0
best_v_dim = 0
best_learning_rate = 0
best_dropout = 0
best_m_target = None
target_attn_poems = None
best_H_target_p = None

for i in range(60):
    
    print("Iteration: {}".format(i+1))
    
    dropout = np.random.uniform(dropout_min, dropout_max)
    learning_rate = 10**np.random.uniform(np.log10(learning_rate_min), np.log10(learning_rate_max))
#     hidden_dim = np.random.randint(hidden_dim_min, hidden_dim_max)
    v_dim = np.random.randint(v_dim_min, v_dim_max)



    m_target_p = BiLSTMTargetNet_withMLP(vocab_size = len(TEXT.vocab), embedding_dim = embedding_dim, 
                                   hidden_dim = hidden_dim_src, label_size=label_size, v_dim = v_dim,
                                   pretrained_vec=TEXT.vocab.vectors, use_gpu = True, dropout = dropout)
    m_target_p.to("cuda")

    opt_t = torch.optim.Adam(filter(lambda p: p.requires_grad, m_target_p.parameters()), learning_rate)

    dev_acc, test_acc, src_attention_p, H_src_p = training_loop_DATN_full(batch_size = batch_size, 
                                                                num_epochs = 10, src_model = m_source_p, 
                                                                tgt_model = m_target_p,
                                                                loss_ = F.kl_div, optim_src = None, optim_tgt = opt_t, 
                                                                training_iter=train_batch, lambda_ = 0.05, 
                                                                dev_iter=valid_batch, test_iter = test_batch, 
                                                                source_flag = True, verbose = True, src_learn_flag = False)

    dropouts.append(dropout)
    learning_rates.append(learning_rate)
    hidden_dims.append(hidden_dim_src)
    v_dims.append(v_dim)
    dev_accuracies.append(dev_acc)
    test_accuracies.append(test_acc)

    try:
        check = dev_accuracies[-1] > dev_accuracies[-2]
    except:
        check = dev_accuracies[-1] > 0

    if check:

        best_dev_acc = dev_acc
        best_learning_rate = learning_rates[-1]
        best_hidden_dim = hidden_dims[-1]
        best_v_dim  = v_dims[-1]
        best_dropout = dropouts[-1]
        best_m_target = m_target_p
        best_test_acc = test_acc
#         target_attn_poems = target_attention
#         best_H_target_p = H_target_p

    df = pd.DataFrame({"droprate": dropouts, "learning_rates": learning_rates, 
                       "hidden_dim": hidden_dims, "v_dim": v_dims, 
                       "dev_accuracy": dev_accuracies, "test_accuracy": test_accuracies})
    df.to_csv("DATN_results_YD.csv")

df = pd.DataFrame({"droprate": dropouts, "learning_rates": learning_rates, 
                   "hidden_dim": hidden_dims, "v_dim": v_dims, 
                   "dev_accuracy": dev_accuracies, "test_accuracy": test_accuracies})

df_2 = pd.DataFrame({"droprate": best_dropout, "learning_rates": best_learning_rate,
                     "hidden_dim": best_hidden_dim, "v_dim": best_v_dim, 
                     "dev_accuracy": best_dev_acc, "test_accuracy": best_test_acc})

df_2.to_csv("DATN_results_target_best_YD.csv")

Iteration: 1
Training...


C:\Users\yashr\Anaconda3\lib\site-packages\torch\nn\functional.py:1992: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
C:\Users\yashr\Documents\Poetry_DATN\functions.py:137: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(tgt_model.parameters(), 5.0)


Evaluating dev...
Evaluating test..
Epoch 0; Loss 2.801196; Dev loss 0.578877; Dev acc: 0.214286; Test acc: 0.297872
Training...
Evaluating dev...
Evaluating test..
Epoch 1; Loss 3.077354; Dev loss 0.515957; Dev acc: 0.093750; Test acc: 0.120567
Training...
Evaluating dev...
Evaluating test..
Epoch 2; Loss 3.771158; Dev loss 0.536134; Dev acc: 0.209821; Test acc: 0.234043
Training...
Evaluating dev...
Evaluating test..
Epoch 3; Loss 2.485437; Dev loss 0.525228; Dev acc: 0.205357; Test acc: 0.244681
Training...
Evaluating dev...
Evaluating test..
Epoch 4; Loss 2.612291; Dev loss 0.541859; Dev acc: 0.205357; Test acc: 0.127660
Training...
Evaluating dev...
Evaluating test..
Epoch 5; Loss 2.651474; Dev loss 0.529813; Dev acc: 0.093750; Test acc: 0.063830
Training...
Evaluating dev...
Evaluating test..
Epoch 6; Loss 2.674862; Dev loss 0.530963; Dev acc: 0.129464; Test acc: 0.152482
Training...
Evaluating dev...
Evaluating test..
Epoch 7; Loss 2.988408; Dev loss 0.526667; Dev acc: 0.267857;

Evaluating test..
Epoch 8; Loss 2.293877; Dev loss 0.580496; Dev acc: 0.142857; Test acc: 0.131206
Training...
Evaluating dev...
Evaluating test..
Epoch 9; Loss 2.661786; Dev loss 0.557889; Dev acc: 0.236607; Test acc: 0.170213
Training...
Evaluating dev...
Evaluating test..
Epoch 10; Loss 2.487230; Dev loss 0.631507; Dev acc: 0.147321; Test acc: 0.124113
Iteration: 7
Training...
Evaluating dev...
Evaluating test..
Epoch 0; Loss 2.412807; Dev loss 0.506284; Dev acc: 0.348214; Test acc: 0.290780
Training...
Evaluating dev...
Evaluating test..
Epoch 1; Loss 2.585972; Dev loss 0.533006; Dev acc: 0.321429; Test acc: 0.386525
Training...
Evaluating dev...
Evaluating test..
Epoch 2; Loss 3.332862; Dev loss 0.508759; Dev acc: 0.303571; Test acc: 0.297872
Training...
Evaluating dev...
Evaluating test..
Epoch 3; Loss 2.956816; Dev loss 0.524665; Dev acc: 0.250000; Test acc: 0.322695
Training...
Evaluating dev...
Evaluating test..
Epoch 4; Loss 3.931747; Dev loss 0.540085; Dev acc: 0.267857; Tes

Training...
Evaluating dev...
Evaluating test..
Epoch 5; Loss 2.715331; Dev loss 0.526091; Dev acc: 0.258929; Test acc: 0.287234
Training...
Evaluating dev...
Evaluating test..
Epoch 6; Loss 2.552318; Dev loss 0.534911; Dev acc: 0.214286; Test acc: 0.358156
Training...
Evaluating dev...
Evaluating test..
Epoch 7; Loss 2.269428; Dev loss 0.528245; Dev acc: 0.133929; Test acc: 0.131206
Training...
Evaluating dev...
Evaluating test..
Epoch 8; Loss 2.522158; Dev loss 0.505724; Dev acc: 0.156250; Test acc: 0.251773
Training...
Evaluating dev...
Evaluating test..
Epoch 9; Loss 2.114169; Dev loss 0.559512; Dev acc: 0.169643; Test acc: 0.265957
Training...
Evaluating dev...
Evaluating test..
Epoch 10; Loss 2.301327; Dev loss 0.551267; Dev acc: 0.142857; Test acc: 0.191489
Iteration: 13
Training...
Evaluating dev...
Evaluating test..
Epoch 0; Loss 2.897138; Dev loss 0.547477; Dev acc: 0.093750; Test acc: 0.078014
Training...
Evaluating dev...
Evaluating test..
Epoch 1; Loss 2.935231; Dev loss 0

...

Training...
Evaluating dev...
Evaluating test..
Epoch 0; Loss 2.882041; Dev loss 0.596777; Dev acc: 0.321429; Test acc: 0.304965
Training...
Evaluating dev...
Evaluating test..
Epoch 1; Loss 2.489127; Dev loss 0.558983; Dev acc: 0.330357; Test acc: 0.319149
Training...
Evaluating dev...
Evaluating test..
Epoch 2; Loss 2.869060; Dev loss 0.524415; Dev acc: 0.236607; Test acc: 0.329787
Training...
Evaluating dev...
Evaluating test..
Epoch 3; Loss 2.333192; Dev loss 0.525007; Dev acc: 0.285714; Test acc: 0.347518
Training...
Evaluating dev...
Evaluating test..
Epoch 4; Loss 3.246388; Dev loss 0.537609; Dev acc: 0.062500; Test acc: 0.081560
Training...
Evaluating dev...
Evaluating test..
Epoch 5; Loss 2.652913; Dev loss 0.540338; Dev acc: 0.285714; Test acc: 0.329787
Training...
Evaluating dev...
Evaluating test..
Epoch 6; Loss 2.723825; Dev loss 0.516563; Dev acc: 0.308036; Test acc: 0.361702
Training...
Evaluating dev...
Evaluating test..
Epoch 7; Loss 2.789824; Dev loss 0.503898; Dev ac

Epoch 8; Loss 2.714374; Dev loss 0.584934; Dev acc: 0.062500; Test acc: 0.095745
Training...
Evaluating dev...
Evaluating test..
Epoch 9; Loss 2.538305; Dev loss 0.612669; Dev acc: 0.084821; Test acc: 0.102837
Training...
Evaluating dev...
Evaluating test..
Epoch 10; Loss 2.586139; Dev loss 0.549716; Dev acc: 0.245536; Test acc: 0.216312
Iteration: 47
Training...
Evaluating dev...
Evaluating test..
Epoch 0; Loss 2.713847; Dev loss 0.521943; Dev acc: 0.058036; Test acc: 0.088652
Training...
Evaluating dev...
Evaluating test..
Epoch 1; Loss 2.750734; Dev loss 0.520496; Dev acc: 0.325893; Test acc: 0.397163
Training...
Evaluating dev...
Evaluating test..
Epoch 2; Loss 2.696030; Dev loss 0.525358; Dev acc: 0.035714; Test acc: 0.078014
Training...
Evaluating dev...
Evaluating test..
Epoch 3; Loss 2.992239; Dev loss 0.511267; Dev acc: 0.258929; Test acc: 0.294326
Training...
Evaluating dev...
Evaluating test..
Epoch 4; Loss 2.520736; Dev loss 0.509007; Dev acc: 0.263393; Test acc: 0.265957
T

Evaluating dev...
Evaluating test..
Epoch 5; Loss 2.461040; Dev loss 0.517026; Dev acc: 0.062500; Test acc: 0.099291
Training...
Evaluating dev...
Evaluating test..
Epoch 6; Loss 2.748502; Dev loss 0.512319; Dev acc: 0.178571; Test acc: 0.269504
Training...
Evaluating dev...
Evaluating test..
Epoch 7; Loss 2.614578; Dev loss 0.513208; Dev acc: 0.174107; Test acc: 0.184397
Training...
Evaluating dev...
Evaluating test..
Epoch 8; Loss 2.769385; Dev loss 0.513317; Dev acc: 0.232143; Test acc: 0.390071
Training...
Evaluating dev...
Evaluating test..
Epoch 9; Loss 2.800626; Dev loss 0.514328; Dev acc: 0.236607; Test acc: 0.315603
Training...
Evaluating dev...
Evaluating test..
Epoch 10; Loss 3.020709; Dev loss 0.503438; Dev acc: 0.321429; Test acc: 0.336879
Iteration: 53
Training...
Evaluating dev...
Evaluating test..
Epoch 0; Loss 3.385186; Dev loss 0.574607; Dev acc: 0.245536; Test acc: 0.301418
Training...
Evaluating dev...
Evaluating test..
Epoch 1; Loss 2.349816; Dev loss 0.519760; Dev

Epoch 1; Loss 2.328716; Dev loss 0.647248; Dev acc: 0.133929; Test acc: 0.180851
Training...
Evaluating dev...
Evaluating test..
Epoch 2; Loss 2.988153; Dev loss 0.538811; Dev acc: 0.120536; Test acc: 0.088652
Training...
Evaluating dev...
Evaluating test..
Epoch 3; Loss 3.337624; Dev loss 0.533913; Dev acc: 0.062500; Test acc: 0.134752
Training...
Evaluating dev...
Evaluating test..
Epoch 4; Loss 3.191878; Dev loss 0.538181; Dev acc: 0.071429; Test acc: 0.070922
Training...
Evaluating dev...
Evaluating test..
Epoch 5; Loss 2.611320; Dev loss 0.516953; Dev acc: 0.147321; Test acc: 0.120567
Training...
Evaluating dev...
Evaluating test..
Epoch 6; Loss 2.172477; Dev loss 0.526172; Dev acc: 0.209821; Test acc: 0.283688
Training...
Evaluating dev...
Evaluating test..
Epoch 7; Loss 3.373095; Dev loss 0.560164; Dev acc: 0.129464; Test acc: 0.134752
Training...
Evaluating dev...
Evaluating test..
Epoch 8; Loss 2.835290; Dev loss 0.544011; Dev acc: 0.267857; Test acc: 0.294326
Training...
Eval

ValueError: If using all scalar values, you must pass an index

In [6]:
df

,droprate,learning_rates,hidden_dim,v_dim,dev_accuracy,test_accuracy
0,0.493469,0.007602,183,17,0.267857,0.223404
1,0.749746,0.007738,183,11,0.200893,0.297872
2,0.405397,0.000465,183,16,0.348214,0.315603
3,0.646440,0.006163,183,15,0.263393,0.326241
4,0.793815,0.000242,183,13,0.281250,0.319149
5,0.349833,0.004270,183,16,0.294643,0.269504
6,0.714568,0.000745,183,10,0.348214,0.290780
7,0.432629,0.000125,183,25,0.321429,0.304965
8,0.608916,0.000354,183,27,0.308036,0.312057
9,0.297347,0.000734,183,19,0.325893,0.304965


In [ ]:
t[0].shape

In [ ]:
F.cosine_similarity(s.squeeze(0), t.squeeze(0), 0)

In [ ]:
t = torch.tensor([1, 2, 3, 4])

In [ ]:
s = torch.tensor([2, 1, 3, 4])

In [ ]:
(t.topk(2).indices == s.topk(2).indices).sum().item()

In [ ]:
lab = torch.tensor([0, 1, 1, 0])

In [ ]:
lab.topk(2)